# Scripts que devuelve las diferencias entre los ficheros

Importamos

In [40]:
import json
import os
import pandas as pd
from difflib import SequenceMatcher
import jellyfish
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from geopy.distance import great_circle
import folium
from folium.plugins import MarkerCluster
from convertbng.util import convert_bng, convert_lonlat, convert_etrs89_to_lonlat
import overpass
import sys

first_arg = sys.argv[1]

In [2]:
print(first_arg)

-f


In [3]:
#Center coordinates
SF_COORDINATES = (40.4168, -3.7038)

# for speed purposes
MAX_RECORDS = 10

In [4]:
def get_id(x):
    try:
        for i in range(len(ids)):
            point = [lat[i],lon[i]]
            if getDistance_meters(x,point) <results[fichero]['distance_SAME']:
                ayu[i] = x
                return ids[i]
        return 'No match'
    except:
        return 'No match'

def get_close(x):
    try:
        for i in range(len(ids)):
            point = [lat[i],lon[i]]
            if getDistance_meters(x,point) <results[fichero]['distance_CLOSE']:
                ayu[i] = x
                return ids[i]
        return 'No match'
    except:
        return 'No match'
    
def get_lat(x):
    try:
        ind = ids.index(x)
        return float(lat[ind])
    except:
        return None

def get_lon(x):
    try:
        ind = ids.index(x)
        return float(lon[ind])
    except:
        return None

def get_name(x):
    try:
        ind = ids.index(x)
        return name[ind]
    except:
        return None

def get_score(x):
    return getDistance_meters([x.lat_OSM, x.lon_OSM],[x.LATITUD, x.LONGITUD])



In [5]:
def getDistance_meters(x,y):
    try:
        return great_circle(x, y).kilometers*1000.0
    except:
        return 0

Creamos un data para los datos de OSM

In [6]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/data/')

In [7]:
indice = pd.read_csv('indice.csv',index_col=0,encoding='latin-1',delimiter=';')

In [8]:
indice.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, BANCOS to MONUMENTOS
Data columns (total 7 columns):
SEARCH            32 non-null object
AYU               32 non-null object
OSM               2 non-null object
DOCTORDATA        32 non-null object
distance_SAME     32 non-null int64
distance_CLOSE    32 non-null int64
palabra           32 non-null object
dtypes: int64(2), object(5)
memory usage: 2.0+ KB


In [9]:
results = {}
for key, df_gb in indice.groupby(indice.index):
    results[str(key)] = df_gb.to_dict('list')

In [10]:
for key in results.keys():
    results[key]['SEARCH'] = results[key]['SEARCH'][0][1:-1]
    results[key]['distance_CLOSE'] = results[key]['distance_CLOSE'][0]
    results[key]['distance_SAME'] = results[key]['distance_SAME'][0]
    results[key]['palabra'] = results[key]['palabra'][0]

In [11]:
results['BANCOS']['SEARCH']

'"amenity"="bench"'

In [12]:
os.listdir()

['.DS_Store',
 '20170614_BANCOS.csv',
 '20170614_PAPELERAS.csv',
 '20171110_InventarioFuentes.csv',
 'anexo i monumentos.xls',
 'artwork.geojson',
 'bench.geojson',
 'drinking_water.geojson',
 'indice.csv',
 'memorial.geojson',
 'monument.geojson',
 'street_lamp.geojson',
 'waste_basket.geojson']

In [13]:
AYU_fichero = '20171110_InventarioFuentes.csv'

In [14]:
fichero = []
for key in results.keys():
    if AYU_fichero.split('_')[1][:-4] == results[key]['palabra']:
        fichero = key

In [15]:
results

{'BANCOS': {'AYU': ['DISTRITO',
   'BARRIO',
   'MODELO',
   'X_ETRS89',
   'Y_ETRS89',
   'Longitud',
   'Latitud'],
  'DOCTORDATA': ['DISTRITO',
   'BARRIO',
   'MODELO',
   'X_ETRS89',
   'Y_ETRS89',
   'LONGITUD',
   'LATITUD'],
  'OSM': [nan, nan, nan, nan, nan, nan, nan],
  'SEARCH': '"amenity"="bench"',
  'distance_CLOSE': 50,
  'distance_SAME': 5,
  'palabra': 'BANCOS'},
 'FUENTES': {'AYU': ['CODIGO',
   'ZONA VERDE / VIA PUBLICA',
   'DISTRITO',
   'DIRECCION',
   'COMPLEMENTO DE DIRECCION',
   'COORD. X',
   'COORD. Y',
   'SISTEMA DE COORDENADAS',
   'ESTADO',
   'OBSERVACIONES',
   'FECHA DATOS',
   'LONGITUD',
   'LATITUD'],
  'DOCTORDATA': ['CODIGO',
   'ZV-VP',
   'DISTRITO',
   'DIRECCION',
   'COMP-DIR',
   'X_ETRS89',
   'Y_ETRS89',
   'SISTEMA DE COORDENADAS',
   'ESTADO',
   'OBSERVACIONES',
   'FECHA',
   'LONGITUD',
   'LATITUD'],
  'OSM': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
  'SEARCH': '"amenity"="drinking_water"',
  'distance_CLOSE

In [16]:
search = 'area["name"="Madrid"];node(area)['+results[fichero]['SEARCH']+'];'

In [17]:
import overpass
api = overpass.API()
OSM = api.Get(search)

In [18]:
OSM['features'][0]['properties']

{'amenity': 'drinking_water'}

Create searching arrays

In [19]:
ids = [OSM['features'][i]['id'] for i in range(len(OSM['features']))]
lat = [OSM['features'][i]['geometry']['coordinates'][1] for i in range(len(OSM['features']))]
lon = [OSM['features'][i]['geometry']['coordinates'][0] for i in range(len(OSM['features']))]
#name = [OSM['features'][i]['properties']['name'] for i in range(len(OSM['features']))]
ayu = ids[:]

Add ayuntamiento data

In [20]:
results[fichero]['DOCTORDATA'][1:]

['ZV-VP',
 'DISTRITO',
 'DIRECCION',
 'COMP-DIR',
 'X_ETRS89',
 'Y_ETRS89',
 'SISTEMA DE COORDENADAS',
 'ESTADO',
 'OBSERVACIONES',
 'FECHA',
 'LONGITUD',
 'LATITUD']

In [21]:
['ZONA VERDE / VIA PUBLICA', 'DISTRITO', 'DIRECCION',
       'COMPLEMENTO DE DIRECCIÓN ', 'COORD. X', 'COORD. Y',
       'SISTEMA DE COORDENADAS', 'ESTADO', 'OBSERVACIONES', 'FECHA DATOS',
       'LONGITUD', 'LATITUD']

['ZONA VERDE / VIA PUBLICA',
 'DISTRITO',
 'DIRECCION',
 'COMPLEMENTO DE DIRECCIÓN ',
 'COORD. X',
 'COORD. Y',
 'SISTEMA DE COORDENADAS',
 'ESTADO',
 'OBSERVACIONES',
 'FECHA DATOS',
 'LONGITUD',
 'LATITUD']

In [22]:
AYU = pd.read_csv(AYU_fichero, encoding='latin-1', delimiter=';',index_col=0)
AYU.columns = [x.upper() for x in AYU.columns.values]
if AYU_fichero == '20171110_InventarioFuentes.csv':
    AYU.columns = results[fichero]['DOCTORDATA'][1:]

try:
    AYU = AYU[AYU.LATITUD != 'Error']
    AYU = AYU[AYU.LONGITUD != 'Error']
except:
    print('No errors!')

AYU.LATITUD = AYU.LATITUD.astype('float32')
AYU.LONGITUD = AYU.LONGITUD.astype('float32')
AYU = AYU.sort_values('LATITUD')
AYU['id_OSM'] = pd.Series()
AYU['lat_OSM'] = pd.Series()
AYU['lon_OSM'] = pd.Series()
AYU['coord_OSM'] = pd.Series()
AYU['name_OSM'] = pd.Series()
AYU['score_OSM'] = pd.Series()
AYU['X_ETRS89'] = AYU['X_ETRS89'].apply(lambda x: float(x.replace(',','.')))
AYU['Y_ETRS89'] = AYU['Y_ETRS89'].apply(lambda x: float(x.replace(',','.')))
AYU['coord'] = list(zip(AYU.LATITUD, AYU.LONGITUD))
#AYU = AYU[:200]
AYU.tail(2)

,ZV-VP,DISTRITO,DIRECCION,COMP-DIR,X_ETRS89,Y_ETRS89,SISTEMA DE COORDENADAS,ESTADO,OBSERVACIONES,FECHA,LONGITUD,LATITUD,id_OSM,lat_OSM,lon_OSM,coord_OSM,name_OSM,score_OSM,coord
CODIGO,,,,,,,,,,,,,,,,,,,
FUE_03-08_0000000120,ZV,FUENCARRAL-EL PARDO,COLONIA MINGORRUBIO 1ª FASE,C/ ALMENAS,433767.8135,4487222.559,ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.782030,40.533100,NaN,NaN,NaN,NaN,NaN,NaN,"(40.5331, -3.78203)"
FUE_03-08_0000000121,ZV,FUENCARRAL-EL PARDO,COLONIA MINGORRUBIO 1ª FASE,C/ ESCUADRÓN,433778.9948,4487403.260,ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.781917,40.534729,NaN,NaN,NaN,NaN,NaN,NaN,"(40.5347, -3.78192)"


In [23]:
AYU.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1598 entries, FUE_06-17_0000000113 to FUE_03-08_0000000121
Data columns (total 19 columns):
ZV-VP                     1598 non-null object
DISTRITO                  1598 non-null object
DIRECCION                 1598 non-null object
COMP-DIR                  1053 non-null object
X_ETRS89                  1598 non-null float64
Y_ETRS89                  1598 non-null float64
SISTEMA DE COORDENADAS    1598 non-null object
ESTADO                    1598 non-null object
OBSERVACIONES             135 non-null object
FECHA                     1598 non-null object
LONGITUD                  1598 non-null float32
LATITUD                   1598 non-null float32
id_OSM                    0 non-null float64
lat_OSM                   0 non-null float64
lon_OSM                   0 non-null float64
coord_OSM                 0 non-null float64
name_OSM                  0 non-null float64
score_OSM                 0 non-null float64
coord                     

In [24]:
AYU.id_OSM = AYU.coord.apply(lambda x: get_id(x)) 
AYU_NO = AYU[AYU.id_OSM == 'No match']
AYU_OK = AYU[AYU.id_OSM != 'No match']

In [25]:
AYU.head(2)

,ZV-VP,DISTRITO,DIRECCION,COMP-DIR,X_ETRS89,Y_ETRS89,SISTEMA DE COORDENADAS,ESTADO,OBSERVACIONES,FECHA,LONGITUD,LATITUD,id_OSM,lat_OSM,lon_OSM,coord_OSM,name_OSM,score_OSM,coord
CODIGO,,,,,,,,,,,,,,,,,,,
FUE_06-17_0000000113,VP,VILLAVERDE,AV. DE LOS ROSALES 407,NaN,443914.6707,4464487.999,ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.660224,40.329044,4330653037,NaN,NaN,NaN,NaN,NaN,"(40.329, -3.66022)"
FUE_06-17_0000000060,ZV,VILLAVERDE,INST. DEP. COL. MARCONI,NaN,440662.8099,4465018.166,ETRS89 / UTM Zona 30N,EN SERVICIO,NaN,3-nov.-17,-3.698551,40.333595,4330662057,NaN,NaN,NaN,NaN,NaN,"(40.3336, -3.69855)"


In [26]:
AYU_CLOSE = AYU_NO.copy()
AYU_CLOSE.id_OSM = AYU_NO.coord.apply(lambda x: get_close(x))
AYU_NO = AYU_CLOSE[AYU_CLOSE.id_OSM == 'No match']
AYU_CLOSE = AYU_CLOSE[AYU_CLOSE.id_OSM != 'No match']

In [27]:
AYU = pd.concat([AYU_OK, AYU_CLOSE, AYU_NO])

In [28]:
AYU.lat_OSM = AYU.id_OSM.apply(lambda x: get_lat(x))
AYU.lon_OSM = AYU.id_OSM.apply(lambda x: get_lon(x))
AYU['coord'] = list(zip(AYU.lat_OSM, AYU.lon_OSM))
AYU.score_OSM = AYU.apply(lambda x: get_score(x), axis=1)
AYU['coord_OSM'] = list(zip(AYU.lat_OSM, AYU.lon_OSM))

try:
    AYU_CLOSE.lat_OSM = AYU_CLOSE.id_OSM.apply(lambda x: get_lat(x))
    AYU_CLOSE.lon_OSM = AYU_CLOSE.id_OSM.apply(lambda x: get_lon(x))
    AYU_CLOSE['coord'] = list(zip(AYU_CLOSE.lat_OSM, AYU_CLOSE.lon_OSM))
    AYU_CLOSE.score_OSM = AYU_CLOSE.apply(lambda x: get_score(x), axis=1)
    AYU_CLOSE['coord_OSM'] = list(zip(AYU_CLOSE.lat_OSM, AYU_CLOSE.lon_OSM))
except:
    print('No close')

AYU_NO.lat_OSM = AYU_NO.id_OSM.apply(lambda x: get_lat(x))
AYU_NO.lon_OSM = AYU_NO.id_OSM.apply(lambda x: get_lon(x))
AYU_NO['coord'] = list(zip(AYU_NO.lat_OSM, AYU_NO.lon_OSM))
AYU_NO.score_OSM = AYU_NO.apply(lambda x: get_score(x), axis=1)
AYU_NO['coord_OSM'] = list(zip(AYU_NO.lat_OSM, AYU_NO.lon_OSM))

In [29]:
AYU = AYU.sort_values('score_OSM')
#AYU.name_OSM = AYU.id_OSM.apply(lambda x: get_name(x))

In [30]:
AYU_MISSING = []
for i in range(len(ids)):
    AYU_MISSING.append((ids[i],lat[i],lon[i]))
        
AYU_MISSING = pd.DataFrame(AYU_MISSING)
AYU_MISSING.columns = ['id_OSM','lat_OSM','lon_OSM']

AYU_MISSING = AYU_MISSING.set_index('id_OSM')
AYU_MISSING['coord'] = list(zip(AYU_MISSING.lat_OSM.values, AYU_MISSING.lon_OSM.values))

In [31]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/csv/')

In [32]:
AYU_CLOSE.to_csv(AYU_fichero[:-4]+'-edit.csv')
AYU_NO.to_csv(AYU_fichero[:-4]+'-missing_OSM.csv')
AYU_MISSING.to_csv(AYU_fichero[:-4]+'-missing_AYU.csv')

## Close to edit

In [33]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/web/')

In [43]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11,max_zoom=20, control_scale = True, prefer_canvas=True)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_CLOSE.id_OSM.values)):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(str(AYU_CLOSE.index.values[i]), parse_html=True),
        location = (AYU_CLOSE.LATITUD.values[i],AYU_CLOSE.LONGITUD.values[i])).add_to(marker_cluster)
map.save(AYU_fichero[:-4]+'-edit.html')

## Missing in OSM

In [35]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11,max_zoom=18, control_scale = True, prefer_canvas=True)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_NO.id_OSM.values[:100])):
    folium.Marker(icon=folium.Icon(color='blue'),popup=folium.Popup(AYU_NO.index.values[i], parse_html=True),
        location = (AYU_NO.LATITUD.values[i],AYU_NO.LONGITUD.values[i])).add_to(marker_cluster)
map.save(AYU_fichero[:-4]+'-missing_OSM.html')

## Missing in Ayu website

In [36]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11,max_zoom=20, control_scale = True, prefer_canvas=True)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_MISSING.index.values)):
    folium.Marker(icon=folium.Icon(color='green'),popup=folium.Popup(AYU_MISSING.index.values[i], parse_html=True),
        location = (AYU_MISSING.lat_OSM.values[i],AYU_MISSING.lon_OSM.values[i])).add_to(marker_cluster)
map.save(AYU_fichero[:-4]+'-missing_AYU.html')